# Hotels Data - Preliminary analysis
- Prepared by: Debananda Sarkar
- Created on: 2020-11-21

## Importing packages

In [1]:
import pandas as pd
import altair as alt
import numpy as np
from sklearn.model_selection import train_test_split

## Import Data

In [2]:
hotels_df = pd.read_csv("../data/raw/hotels_dataset.csv")

## Setting global seed

In [3]:
seed = 2020

## Splitting training and test data

In [4]:
train_df, test_df = train_test_split(hotels_df, test_size=0.2, random_state=seed)

## Checking class imbalance

In [5]:
class_dist = train_df.value_counts("is_canceled").reset_index(name="count")
class_dist["perc"] = np.round(class_dist["count"]/train_df.shape[0] * 100, 2)
class_dist

,is_canceled,count,perc
0,0,60105,62.93
1,1,35407,37.07


In [6]:
class_dist_plot = alt.Chart(class_dist, title="Class Distribution").mark_bar().encode(
    x = alt.X("perc", scale=alt.Scale(domain= (0, 100)), title = "Percentage of observations"),
    y = alt.Y("is_canceled:N", title="Class value"),
    text = "perc"
)
class_dist_plot + class_dist_plot.mark_text(dx = 18)

alt.LayerChart(...)

### Observations

- We have a class imbalance between positive (where booking was canceled) and negative (where booking was not canceled) class
- We would like to predict the cancellation as accurately as possible, so that the hotel does not get a unwanted surprise. Hence, we would like to maximize recall
- However, in the process of maximizing recall we might over predict cancellation. That will be an adverse scenario as the management may get into panic and start introducing promotions which might take a toll of hotel revenue. Hence, we would like to keep the precision high as well.
- As we are interested in keeping both precision and recall high, **f1-score** will be a good evaluation metric here.

## Checking correlation among all variables

In [7]:
corr_df = train_df.corr().stack().reset_index(name="corr")
corr_df["round_corr"] = np.round(corr_df["corr"], 2)
corr_plot = (
    alt.Chart(corr_df, title="Feature Correlation")
    .mark_rect()
    .encode(
        x="level_0",
        y="level_1",
        tooltip="corr",
        color=alt.Color("corr", scale=alt.Scale(domain=(-1, 1), scheme="purpleorange"))
    )
    .properties(width=500, height=500)
)
corr_plot

corr_text = (
    alt.Chart(corr_df)
    .mark_text(size=8)
    .encode(
        x="level_0",
        y="level_1",
        text="round_corr",
    )
    .properties(width=500, height=500)
)
corr_plot + corr_text

alt.LayerChart(...)

### Observations:

- We have moderate correlation between
    - `arrival_date_week_number` and `arrival_date_year`
    - `stay_in_week_nights` and `stay_in_weekend_nights`
    
- We need to explore these relation further

## Correlation with target

In [8]:
corr_plot = (
    alt.Chart(corr_df[corr_df.level_1 == "is_canceled"], title="Feature Correlation")
    .mark_rect()
    .encode(
        x="level_0",
        y="level_1",
        tooltip="corr",
        color=alt.Color("corr", scale=alt.Scale(domain=(-1, 1), scheme="purpleorange"))
    )
    .properties(width=600)
)
corr_plot

corr_text = (
    alt.Chart(corr_df[corr_df.level_1 == "is_canceled"])
    .mark_text(size=8)
    .encode(
        x="level_0",
        y="level_1",
        text="round_corr",
    )
    .properties(width=600)
)
corr_plot + corr_text

alt.LayerChart(...)

### Observation

- `lead_time` and `total_of_special_requests` have some correlation with the target.

## Checking null values

In [9]:
null_df = train_df.isna().sum().reset_index(name="count_of_nulls").query("count_of_nulls != 0")
null_df["perc"] = np.round(null_df["count_of_nulls"] / train_df.shape[0] * 100, 2)
null_df

,index,count_of_nulls,perc
10,children,4,0.00
13,country,383,0.40
23,agent,13101,13.72
24,company,90071,94.30


### Observation

- We can drop `company` as 94% values are null

## Feature Categorization

In [10]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95512 entries, 74159 to 41824
Data columns (total 32 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   hotel                           95512 non-null  object 
 1   is_canceled                     95512 non-null  int64  
 2   lead_time                       95512 non-null  int64  
 3   arrival_date_year               95512 non-null  int64  
 4   arrival_date_month              95512 non-null  object 
 5   arrival_date_week_number        95512 non-null  int64  
 6   arrival_date_day_of_month       95512 non-null  int64  
 7   stays_in_weekend_nights         95512 non-null  int64  
 8   stays_in_week_nights            95512 non-null  int64  
 9   adults                          95512 non-null  int64  
 10  children                        95508 non-null  float64
 11  babies                          95512 non-null  int64  
 12  meal                        

In [11]:
train_df.head(10).T

,74159,59282,56463,94823,247,35268,16557,4757,1445,86090
hotel,City Hotel,City Hotel,City Hotel,City Hotel,Resort Hotel,Resort Hotel,Resort Hotel,Resort Hotel,Resort Hotel,City Hotel
is_canceled,1,1,1,0,1,0,1,0,0,0
lead_time,349,254,174,169,69,68,247,169,2,156
arrival_date_year,2015,2016,2016,2016,2015,2017,2015,2016,2015,2016
arrival_date_month,October,October,September,August,July,April,October,March,August,March
arrival_date_week_number,40,44,37,32,28,15,41,14,36,13
arrival_date_day_of_month,1,25,7,5,9,13,9,31,30,23
stays_in_weekend_nights,0,0,0,2,2,1,1,0,1,0
stays_in_week_nights,2,5,1,4,6,3,2,3,0,4
adults,2,2,2,2,2,2,2,2,2,2


In [12]:
numeric_features = [
    "lead_time",
    "stays_in_weekend_nights",
    "stays_in_week_nights",
    "adults",
    "children",
    "babies",
    "previous_cancellations",
    "previous_bookings_not_canceled",
    "booking_changes",
    "days_in_waiting_list",
    "adr",
    "required_car_parking_spaces",
    "total_of_special_requests"
]
categorical_features = [
    "hotel",
    "arrival_date_month",
    "meal",
    "country",
    "market_segment",
    "distribution_channel",
    "reserved_room_type",
    "deposit_type",
    "customer_type"
]
drop_features = ["company", "reservation_status", "reservation_status_date"]
remainder_feature = [
    "arrival_date_year",
    "arrival_date_week_number",
    "arrival_date_day_of_month",
    "agent"
]
binary_features = ["is_repeated_guest"]